<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
<img src="images/logos/nsidc_daac.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 4

## Using [pandas](http://pandas.pydata.org/) for analysis.

### Getting acquainted with our data

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

Let's find a nice time series to examine.

David Robinson's Rutgers Northern Hemisphere Snow Cover Extent is a coarse (88 x 88) northern-hemisphere grid, with **weekly** data going back to 1966.

http://climate.rutgers.edu/snowcover/docs.php?target=datareq

_Robinson, David A., Estilow, Thomas W., and NOAA CDR Program (2012):NOAA Climate Date Record (CDR) of Northern Hemisphere (NH) Snow Cover Extent (SCE), Version 1. [indicate subset used]. NOAA National Climatic Data Center. doi:10.7289/V5N014G9 [access date]._

We have already downloaded this dataset and included it, zipped, with the tutorial files. Let's unzip the file and then open the NetCDF file:

In [ ]:
%%bash
rm -f nhsce_v01r01_19661004_20160201.nc
unzip $PWD/data/nhsce_v01r01_19661004_20160201.nc.zip

In [ ]:
import netCDF4
ds = netCDF4.Dataset('nhsce_v01r01_19661004_20160201.nc')

Examine the dataset's title:

In [ ]:
ds.title

And its NetCDF attributes:

In [ ]:
ds.ncattrs()

List the dataset's variables:

In [ ]:
for variable in ds.variables:
    print(variable)

Let's attach some Python variables to some of the dataset's metadata variables:

In [ ]:
area = ds.variables['area']
land = ds.variables['land']
latitude = ds.variables['latitude']
longitude = ds.variables['longitude']

We know from the dataset's title that this is northern-hemisphere data, but what does the grid really look like? Let's plot just the land points as a quick sanity check:

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(land[:, :], cmap='Accent', interpolation='nearest')

If you are accustomed to looking at polar projections (and if you squint), you can probably see North America on the lower part of the grid. (The `module-4-extra` notebook in the `extra` folder shows how to improve this plot a bit.)

Let's attach to the dataset's snow cover variable and get an idea of its contents:

In [ ]:
snowcover = ds.variables['snow_cover_extent']
snowcover

So, our `snowcover` variable comprises 2574 88 x 88 grids of {0,1} values where `1 = snow_covered` and `0 = no_snow`.

Now we'll extract all the data from the URL into a local Python variable. The colons in the array slice are shorthand for "all the elements in this dimension":

In [ ]:
all_data = snowcover[:, :, :]

Now we can do a simple plot of the data, and use an interactive widget to show one grid at a time, moving through time via the slider control:

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

@interact(index=widgets.IntSlider(min=0, max=2573, step=4, value=0, continuous_update=False))
def show_it(index=0):
    plt.figure(figsize=(10, 10))
    plt.imshow(all_data[index, :, :], interpolation='nearest', cmap='Blues')

So, we have a series of snow / no-snow binary grids.

Previously, we also extracted an `area` grid from our dataset. Areas on 2D maps do not necessarily represent, in all locations, equal areas on the represented 3D surface (i.e. the Earth), so the `area` grid tells us how many physical km<sup>2</sup> each of our grid's cells represents. Let's visualize this information as a colormap:

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(area, interpolation='nearest', cmap="plasma")
cb = plt.colorbar()
cb.set_label('Grid Cell Area: $km^2$')
print(area[:].max())
print(area[:].min())

Now we can use NumPy multiplication to scale the snow-covered cells by their corresponding areas and visualize the result:

In [ ]:
@interact(index=widgets.IntSlider(min=0, max=2573, step=4, value=0, continuous_update=False))
def show_it(index=0):
    plt.figure(figsize=(10, 10))
    plt.imshow(all_data[index, :, :] * area[:], interpolation='nearest', cmap='plasma')

We'd like to calculate the total snow-covered surface area in the northern hemisphere at a given time. We could count the number of grid cells with snow cover, but that doesn't give us an area.

We need to multiply each grid cell by its area, **then** sum them over the whole grid. Our `area` array provides a km<sup>2</sup> area for each grid cell.

Let's define a quick routine to compute each grid's total snow-covered area:

In [ ]:
def snowcover_area_km2(grid, area):
    return np.sum(grid * area)

And let's compute each weekly total snow-covered area in km<sup>2</sup>:

In [ ]:
weeks = all_data.shape[0]                             # The number of weeks in our data set
total_area = np.ma.empty(weeks)                       # An empty NumPy masked array with as many elements as weeks
for i, grid in enumerate(all_data):                   # Loop over all the grids and, for each...
    total_area[i] = snowcover_area_km2(grid, area[:]) # ...set a total_area element with the km^2 snow-covered area

These are time-dependent data, so let's get our hands on time data matching the snow cover data.

Our dataset provides a `time` variable:

In [ ]:
ds.variables['time']

Let's extract the `time` variable, then use the NetCDF `num2date` helper function to convert the provided values into Python `datetime` objects based on the `days since 1966-10-03` string:

In [ ]:
ds_time = ds.variables['time']
times = netCDF4.num2date(ds_time[:], ds_time.units)

Compare the original, basic integer values to the feature-rich `datetime` objects we got:

In [ ]:
print(ds_time[3:7])
print(times[3:7])

Note that the values in `ds_time` are seven days apart, as we'd expect from weekly data.

With corresponding area and time data, we can now plot snow-covered area vs time:

In [ ]:
plt.figure(figsize=(15, 2))
plt.plot(times, total_area)
plt.title('NH Weekly Snow-Covered Area in $km^2$')

This looks like sensible seasonally oscillating snow-covered area in the northern hemisphere.

### Working with pandas Series

Pandas provides lots of routines for working with time-series data. In particular, we have [pandas.Series](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html):

_[Series](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#series) is a one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). The axis labels are collectively referred to as the index. The basic method to create a Series is to call:_

      s = pd.Series(data, index=index)

Let's import pandas and create a `Series` object from our snow cover and associated time data:

In [ ]:
import pandas as pd
ts = pd.Series(total_area, index=times)

The `Series.head()` function lets us examine the first few items from a series:

In [ ]:
ts.head()

`Series` has a built-in `plot()` method that gives us something like our original data, plotting the data values vs the index:

In [ ]:
plt.figure(figsize=(15, 2))
ts.plot(title='NH Snow-Covered Area in $km^2$')

Examine the index of our `Series`:

In [ ]:
ts.index

So, our `Series`' index is a pandas `DatetimeIndex`. The [documentation for `DatetimeIndex`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html) is pretty overwhelming, so let's just look at a few things we can do.

We can use built-in attributes that know about years, months, etc.:

In [ ]:
ts.index.year

In [ ]:
ts.index.month

In [ ]:
ts.index.dayofyear

We can select data based on the index. Here, again looking at the full time series, we select, and then plot, data that fell in the month of December, across all years:

In [ ]:
ts[ts.index.month == 12].head(10)

In [ ]:
ts[ts.index.month == 12].plot(style='.',
                              title='NH December Snow-Covered Area in $km^2$',
                              ylim=(0, 6e7))

In [ ]:
ts[ts.index.year == 2000].plot(title='NH Snow-Covered Area in $km^2$: Year 2000')

What was the maximum value across the _entire_ time series?

In [ ]:
ts.max()

On what day did that maximum occur?

In [ ]:
ts[ts == ts.max()]

How often (and when) did snow-covered area reach at least 95% of its all-time maximum? That is, to what extent was the all-time maximum an outlier?

In [ ]:
ts[ts >= ts.max() * .95]

## Resampling time series 

Let's say we need to compare our total snow-covered area with some other geophysical constant, one that is expressed as monthly values. We'll need to transform our weekly data into monthly data.

We can use the [`Series.resample()`](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.Series.resample.html) method to "align" our data with months.

In [ ]:
ts.resample('MS').mean().head()

`'MS'` in the above statement means **M**onthly at **S**tart of month -- without the `S`, the dates would align to the last day of each month.

Monthly resampling averages all of the 7-day periods that start in a particular month, producing a single representative value for that month.

We can plot a somewhat arbitrary 50-week subset of the resampled time series, comparing it (in blue), with the original weekly data (in green):

In [ ]:
plt.figure(figsize=(15, 2))
ts[200:250].resample('MS').mean().plot(marker='x')
ts[200:250].plot(marker='.')
plt.title('50 Weeks of NH Snow-Covered Area in $km^2$')

Or, we can look at 50 consecutive months (a little more than 4 years) of resampled data:

In [ ]:
plt.figure(figsize=(15, 2))
ts.resample('MS').mean()[100:150].plot(title='50 Months of Snow-Covered Area in $km^2$')

But we have a problem.

In this dataset, each indexed snow cover grid represents a week of data. So the grid with index '1966-10-31' represents the _week_ beginning on that day _through_ 1966-11-06, and some (most!) of the days in that week actually fall in November and should contribute to November's mean, not October's.

In [ ]:
ts.head()

We'd like to break our weekly data into days so that e.g. October and November days fall into their respective months. If we do naive **D**aily resampling, note the missing dates between 10-31 and 11-07:

In [ ]:
ts.resample('D').mean()[20:29]

Let's resample again, this time using pandas' `ffill()` feature to **f**orward **fill** the missing dates between each week-start date with the provided value. For example, `ffill()` will create the missing dates 1996-11-01 through 1966-11-06 with the value provided for the week starting 1996-10-31.

In [ ]:
ts.resample('D').ffill(limit=7)[20:29]

So, we succeeding in filling in the missing days with a value copied from the preceding week-start value.

Now when we resample to monthly, we've correctly weighted all of the data for a particular month:

In [ ]:
ts.resample('D').ffill(limit=7).resample('MS').mean().head()

So far, we haven't been saving the results of our resampling, but now let's save our best effort to a variable so we can work more with it:

In [ ]:
monthly = ts.resample('D').ffill(limit=7).resample('MS').mean()

Let's plot values, over the entire time series, for a couple of semi-randomly-selected months:

In [ ]:
plt.figure(figsize=(15, 3))
monthly[monthly.index.month == 2].plot(linestyle='-', label='February', legend=True)
monthly[monthly.index.month == 5].plot(marker='.', label='May', legend=True, title='Feb vs May')

No surprises there!

We can answer the question: "Which year's February had the lowest snow-covered area?"

In [ ]:
monthly[monthly.index.month == 2].idxmin().year

Or: "What are the rankings of March snow-covered area from greatest to least?"

In [ ]:
marches = monthly[monthly.index.month ==  3]
rank = marches.rank(ascending=False)
rank.head()

The ranks are given in the second column, and the March with the highest snow-covered area would be the one with rank 1.

In [ ]:
rank[rank == 1.]

To determine the actual snow-covered area for that March, we can query our `march` time series using the index value that we now know:

In [ ]:
marches['1978-03-01'] # pandas converts string -> datetime object automatically

### Working with pandas DataFrame

Now let's take a look at [`pandas.DataFrame`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)  

     Two-dimensional size-mutable, potentially heterogeneous tabular data
     structure with labeled axes (rows and columns). Arithmetic operations
     align on both row and column labels. Can be thought of as a dict-like
     container for Series objects. The primary pandas data structure.

We can use a `DataFrame` to align March rank and snow cover values into a single object.

Let's create a simple `DataFrame` from our monthly data using March and its associated ranks. The `marches` series contains the actual snow cover values, and the `rank` series contains the rankings but, if we look at the heads of both series, we see that the indices -- date objects representing the first of March for a range of years -- are the same:

In [ ]:
marches.head()

In [ ]:
rank.head()

This makes it trivial to create a `DataFrame` comprising both series. We give the `DataFrame` constructor the series we want to use along with labels for those series:

In [ ]:
df_ranked_snowcover = pd.DataFrame(data={'snowcover': marches, 'rank': rank})
df_ranked_snowcover.head()

The columns of the `DataFrame` have the names we specified in the call to the constructor. Note that, while _rank_ was a good name for the data contained in the `rank` series, the data in the `marches` series needed a better name.

We can use the column names to refer to the series and, for example, sort the data based on the "rank" column:

In [ ]:
df_ranked_snowcover.sort_values('rank', ascending=True).head()

It's nice to be able to use named columns instead of trying to remember numeric indices as one might in, for example, pure NumPy. Here we get the same result, in a single statement, as we obtained previously by making separate queries on the original snow cover and rank series.

Say that we now want to compute the anomalies from the mean for all Marches across the time series. With a `DataFrame`, we can simply add a new column representing the difference between the all-Marches mean from each value:

In [ ]:
mean = df_ranked_snowcover['snowcover'].mean() # this is a scalar value
df_ranked_snowcover['anomaly'] = (df_ranked_snowcover['snowcover'] - mean)
df_ranked_snowcover.head()

Let's go back to our `monthly` pandas `Series` and convert it into a `DataFrame` to investigate what we can do with `DataFrame` indices.

In [ ]:
df_snowcover0 = pd.DataFrame(monthly, columns=['snowcover'])
df_snowcover0.head()

This doesn't look so different from the `Series` object, but now that we're working with a `DataFrame`, we can augment the index, creating a so-called "multi-index":

In [ ]:
df_snowcover1 = df_snowcover0.set_index([df_snowcover0.index.year, df_snowcover0.index.month])
df_snowcover1.head(16)

Our new `MultiIndex` has two levels: years and months.

We can select data from this index using the [`loc()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html) method on `DataFrame`:

In [ ]:
df_snowcover1.loc[1979:1980] # pandas slices have an inclusive upper bound!

Or we can use a cross section (the [`xs()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.xs.html) method on `DataFrame`) to grab a specific month (May=5) and display the results by year:

In [ ]:
df_snowcover1.xs(5, level=1).head()

We can use [`unstack()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) on the months level to get an index of years, with columns of months. This pulls the specificed index (level 1 is month) out and spreads it as columns under the existing _snowcover_ column index:

In [ ]:
year_by_month = df_snowcover1.unstack(level=1)
year_by_month.head(3)

If we prefer rows of months and columns of years, we could unstack level 0:

In [ ]:
month_by_year = df_snowcover1.unstack(level=0)
month_by_year.head(3)

We can still select from the `DataFrame`, first selecting the _snowcover_ level (which contains all the years) and then subsetting by year...

In [ ]:
month_by_year['snowcover'][[1970, 1980, 1990, 2000, 2010]]

...and plotting these data:

In [ ]:
month_by_year['snowcover'][[1970, 1980, 1990, 2000, 2010]].plot()

We can save our work by writing the monthly data out to a .csv file:

In [ ]:
monthly.name = 'snowcover'
monthly.to_csv('monthly-extents.csv', index_label='date', header=True)

In [ ]:
%%bash
head monthly-extents.csv

In Module 5, we'll do some more plotting, as well as calculation of a least-squares fit line, and look briefly at the xarray library.